# Setup

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm


# Business Understanding

# Data Understanding

## Collect initial data

Daten wurde von Kunden bereitgestellt.

In [99]:
# read data from provided csv
original_data = pd.read_csv(
    "train.csv",
    sep="$",
    true_values=["Yes", "yes", "1"],
    false_values=["No", "no", "0"],
    index_col=False,
    low_memory=False,
)


Probleme:
Das CSV ist nicht wohgeformt und kann daher nicht ohne Weiteres eingelesen werden. Die erste Zeile endet mit dem Separatorzeichen ($)

## Analyse der Originaldaten

In [47]:
original_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380999 entries, 0 to 380998
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            380999 non-null  int64  
 1   id                    380999 non-null  int64  
 2   Gender                379948 non-null  object 
 3   Age                   370107 non-null  object 
 4   Driving_License       380948 non-null  object 
 5   Region_Code           380999 non-null  object 
 6   Previously_Insured    380948 non-null  object 
 7   Vehicle_Age           380948 non-null  object 
 8   Vehicle_Damage        380948 non-null  object 
 9   Annual_Premium        380999 non-null  float64
 10  Policy_Sales_Channel  380999 non-null  object 
 11  Vintage               380948 non-null  object 
 12  Response              380999 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(9)
memory usage: 35.2+ MB


In [5]:
original_data.head(20)


,Unnamed: 0,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,1,Male,44,False,28.0,False,> 2 Years,True,40454.0,26.0,217,True
1,1,2,Male,76,False,3.0,False,1-2 Year,False,33536.0,26.0,183,False
2,2,3,Male,47,False,28.0,False,> 2 Years,True,38294.0,26.0,27,True
3,3,4,Male,21,True,11.0,True,< 1 Year,False,28619.0,152.0,203,False
4,4,5,Female,29,True,41.0,True,< 1 Year,False,27496.0,152.0,39,False
5,5,6,Female,24,False,33.0,False,< 1 Year,True,2630.0,160.0,176,False
6,6,7,Male,23,False,11.0,False,< 1 Year,True,23367.0,152.0,249,False
7,7,8,Female,56,False,28.0,False,1-2 Year,True,32031.0,26.0,72,True
8,8,9,Female,24,True,3.0,True,< 1 Year,False,27619.0,152.0,28,False
9,9,10,Female,32,True,6.0,True,< 1 Year,False,28771.0,152.0,80,False


### Erste Erkenntnisse
1. 3 der Spalten mit augenscheinlich booleschen Werten konnten nicht automatisch in den boolesche Datentypen übertragen werden. Also müssen hier invalide oder fehlende Werte vorliegen
1. Die Spalte "Age" wurde nicht, wie erwartet in Integer oder Float umgewandelt, auch hier müssen falsche oder fehlende Werte zu finden sein.
1. Die Spalte "Vehicle_Age" ist eine Klassenvariable, satt wie zu erwarten numerisch.
1. Die erste Spalte beinhaltet nur einen Identifikator, der um -1 zur Spalte "Id" versetzt ist.

In [8]:
original_data.describe(include="all").transpose()
# keine spannenden Erkenntnisse?


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,380999,NaN,NaN,NaN,190499,109985,0,95249.5,190499,285748,380998
id,380999,NaN,NaN,NaN,190500,109985,1,95250.5,190500,285750,380999
Gender,379948,2,Male,205447,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,370107,220,24,25228,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Driving_License,380948,2,False,206635,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Region_Code,380999,54,28.0,106372,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Previously_Insured,380948,2,False,206635,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vehicle_Age,380948,3,1-2 Year,200228,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vehicle_Damage,380948,2,True,192328,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Annual_Premium,380999,NaN,NaN,NaN,30527.7,17243,-9997,24371,31656,39390,540165


### Untersuchung, warum die Spalten nicht mit den vermeindlich korrekten Datentypen versorgt werden
Ausgabe aller einzigartigen Werte. So kann die Skala bestimmt und Eingabefehler gefunden werden.

In [38]:
arr = original_data["Age"].unique()
arr


array(['44', '76', '47', '21', '29', '24', '23', '56', '32', '41', '71',
       '37', '25', '42', '60', '65', '49', '34', '51', '26', '57', '79',
       '48', '45', '72', '30', '54', '27', '38', '22', '78', '20', '39',
       '62', '58', '59', '63', '50', '67', '77', '28', '69', '52', '31',
       '33', '43', '36', '53', '70', '46', '55', '40', '61', '75', '64',
       '35', '66', '68', '74', '73', '84', '83', '81', '80', '133', '171',
       '163', '144', '187', '203', '143', '123', '183', '167', '129',
       '127', '142', '82', '175', '128', '150', '196', '154', '198',
       '116', '152', '161', '114', '166', '124', '134', '173', '106',
       '199', '162', '157', '132', '156', '119', '159', '85', '192',
       '201', '177', '121', '160', '136', '148', '158', '113', '184',
       '182', '122', '190', '174', '176', '195', '147', '189', nan, '181',
       '188', '107', '145', '149', '137', '139', '126', '130', '202',
       '138', '193', '109', '179', '155', '125', '172', '200', '135

Man sieht, dass es bei den letzten Werten Eingabefehler gab. Sie enthalten realisitische Zahlenwerte, aber auch 2 Punkte. Also müssen die Punkte entfernt werden, bevor der Datentyp umgewandelt werden kann.

In [108]:
original_data["Age"] = original_data["Age"].str.replace(".", "")
original_data["Age"] = original_data["Age"].astype(pd.StringDtype())
print(original_data["Age"].dtype)
original_data["Age"] = original_data["Age"].astype(pd.Int16Dtype())

AttributeError: Can only use .str accessor with string values!

In [111]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380999 entries, 0 to 380998
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            380999 non-null  int64  
 1   id                    380999 non-null  int64  
 2   Gender                379948 non-null  object 
 3   Age                   370107 non-null  Int16  
 4   Driving_License       380948 non-null  object 
 5   Region_Code           380999 non-null  object 
 6   Previously_Insured    380948 non-null  object 
 7   Vehicle_Age           380948 non-null  object 
 8   Vehicle_Damage        380948 non-null  object 
 9   Annual_Premium        380999 non-null  float64
 10  Policy_Sales_Channel  380999 non-null  object 
 11  Vintage               380948 non-null  object 
 12  Response              380999 non-null  bool   
dtypes: Int16(1), bool(1), float64(1), int64(2), object(8)
memory usage: 33.4+ MB


# Data Preparation

# Modeling

# Evaluation

# Deployment